In [ ]:
from makecard import drawSvg
import re
import retrieve_image

In [ ]:
from collections import Counter
import community
import itertools
import matplotlib.pyplot as plt
import networkx as nx
from nltk.corpus import wordnet
import random
import re
import spacy
from textblob import TextBlob

'''
Parse keywords
'''

countpath = 'theresttext'
regex = r'\(\'([a-z]+)\', ([0-9]+)\)'

wordCounter = Counter()

with open(countpath, 'r') as f:
	line = f.readline()
	foundall = re.findall(regex, line)
	for match in foundall:
		wordCounter[match[0]] = int(match[1])

sentimentspath = 'sentimentstext'
regex = r'([a-z]+) Sentiment\(polarity=(-?[0-9]+\.[0-9]+), subjectivity=(-?[0-9]+\.[0-9]+)\)'

all = []
suit1 = [] # neg polarity # <0.5 subjectivity
suit2 = [] # neg polarity # >=0.5 subjectivity
suit3 = [] # pos polarity # <0.5 subjectivity
suit4 = [] # pos polarity # >=0.5 subjectivity

with open(sentimentspath, 'r') as f:
	line = f.readline()
	while line:
		m = re.match(regex, line)
		(word, polarity, subjectivity) = m.groups()
		p = float(polarity)
		s = float(subjectivity)
		
		if p < 0 and s < 0.5:
			suit1.append(word)
		elif p < 0 and s >= 0.5:
			suit2.append(word)
		elif p >= 0 and s >= 0.5:
			suit3.append(word)
		elif p >= 0 and s < 0.5:
			suit4.append(word)
		
		all.append(word)
		line = f.readline()
		

# use the elements for suits, as is customary
earthsyns = wordnet.synsets('earth', pos = wordnet.NOUN)
watersyns = wordnet.synsets('water', pos = wordnet.NOUN)
airsyns = wordnet.synsets('air', pos = wordnet.NOUN)
firesyns = wordnet.synsets('fire', pos = wordnet.NOUN)

elesyns = [earthsyns, watersyns, airsyns, firesyns]
suits = [suit1, suit2, suit3, suit4]
afinities = []

for suit in suits:
	# afinities for earth, water, air, and fire, in order
	sumAfinities = [0, 0, 0, 0]

	lensuit = 0
	for word in suit:
		lensuit += wordCounter[word]

	for i, word in enumerate(suit):
		for syn1 in wordnet.synsets(word, pos = wordnet.NOUN):
			for i, ele in enumerate(elesyns):
				maxMatch = 0
				for synE in ele:
					newMatch = syn1.lch_similarity(synE) 
					if newMatch > maxMatch:
						maxMatch = newMatch

				if maxMatch > 2 and wordCounter[word] > 10:
					sumAfinities[i] += maxMatch * wordCounter[word]
    
	sumAfinities = [x/lensuit for x in sumAfinities]
	afinities.append(sumAfinities)
# print (afinities)
    
options = list(range(4))
for i, suit in enumerate(afinities):
	index = suit.index(max([suit[x] for x in options]))
	options.remove(index)
# 	print(index)

In [ ]:
'''
Create graph
'''

G = nx.Graph()
for i in range(len(all)):
	G.add_node(all[i])
        
for i in range(len(all)):
	for j in range(i+1, len(all)):
		maxMatch = 0
		for syn1 in wordnet.synsets(all[i], pos = wordnet.NOUN):
			for syn2 in wordnet.synsets(all[j], pos = wordnet.NOUN):
				newMatch = syn1.lch_similarity(syn2)
				if newMatch > maxMatch:
					maxMatch = newMatch

		G.add_edge(all[i], all[j], weight=maxMatch)

In [ ]:
def getsuit(word):
    suit = 1
    if word in suit2:
        suit = 2
    elif word in suit3:
        suit = 3
    elif word in suit4:
        suit = 4
    return suit

In [ ]:
'''
Create processed graph
'''

G2 = nx.Graph()
for node in G.nodes():
    suit = getsuit(node)
    G2.add_node(node, suit=suit)
    
for node in G.nodes():
    for node2 in G.neighbors(node):
        weight = G.get_edge_data(node, node2)['weight']
        if weight > 3:
            G2.add_edge(node, node2, weight=weight)

In [ ]:
'''
Make card
'''
def makesuitcard(suit, num, keyword):
    width = 183
    height = 309

    polygons, foundimg = retrieve_image.getImage(keyword, desired_width=width, desired_height=height, return_found=True)
    path = 'cards/suits/{}_{}.svg'.format(suit, num)
    drawSvg(width, height, polygons, keyword, '{} of {}'.format(num, suit), path)

    foundimg_path = 'found_imgs/suits/{}_{}.jpg'.format(suit, num)
    foundimg.save(foundimg_path)

In [ ]:
'''
Write to booklet
'''
def writebooklet(descr_suits):
    with open('booklet.md', 'a') as f:
        f.write('## Minor Arcana\n\n')
        for descr_suit in descr_suits:
            for descr in descr_suit:
                f.write(descr)

In [ ]:
''' 
Get suit card assignments from graph partition
'''
parts = list(nx.connected_components(G2))
selected = []
flatselected = []
for part in parts:
    if len(list(part)) > 1:
        selected.append(part)
        flatselected.extend(list(part))

selected = sorted(selected, key=lambda x: len(x), reverse=True)

cardnums = list(range(1, 10+1))
cardnums.extend(['page', 'knight', 'queen', 'king'])
suitnums = list(range(1, 4+1))
    
descr_suits = [[], [], [], []]
make_list = []

for num in cardnums:
    for suit in suitnums:
        for s in selected:
            options = [x for x in s if getsuit(x)]
            if len(options) > 0:
                card = random.choice(options)
#                 print('{} {} {}'.format(suit, num, card))

                # these are variables because we may want to call them other names: 
                # such as pentacle, chalice, sword, wand
                earth = 'earth'
                water = 'water'
                air = 'air'
                fire = 'fire'
                suits = [earth, water, air, fire]
                suitname = suits[suit-1]
                make_list.append((suitname, num, card))

                descr = '**{} of {}**\n'.format(num, suitname)
                descr += ', '.join(list(s))
                descr += '\n\n'
                descr_suits[suit-1].append(descr)
                selected.remove(s)
                break

writebooklet(descr_suits)

In [ ]:
'''
Call make card
'''
for i, (suitname, num, card) in list(enumerate(make_list)):
    makesuitcard(suitname, num, card)

In [ ]:
G3 = G2.subgraph(flatselected)
sortednodes = sorted(G3.nodes(), key=lambda x: getsuit(x))
matrix = nx.adjacency_matrix(G2, nodelist=sortednodes)

In [ ]:
fig, ax = plt.subplots(figsize=(17,17))
ax.matshow(matrix.toarray())
fig.show()